# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234274307830                   -0.50    8.0
  2   -7.249896893972       -1.81       -1.38    1.0
  3   -7.250576864996       -3.17       -1.71    2.0
  4   -7.250936136746       -3.44       -1.86    1.0
  5   -7.251331906477       -3.40       -2.78    1.0
  6   -7.251338263726       -5.20       -3.17    3.0
  7   -7.251338730790       -6.33       -3.66    1.0
  8   -7.251338792274       -7.21       -4.38    1.0
  9   -7.251338798148       -8.23       -4.67    3.0
 10   -7.251338798652       -9.30       -5.19    1.0
 11   -7.251338798691      -10.41       -5.50    2.0
 12   -7.251338798703      -10.90       -5.96    1.0
 13   -7.251338798704      -11.94       -6.25    2.0
 14   -7.251338798704      -12.72       -6.81    1.0
 15   -7.251338798705      -13.43       -7.14    2.0
 16   -7.251338798705      -14.15       -7.62    1.0
 17   -7.251338798705      -14.75       -7.66 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04800132112501075
[ Info: Arnoldi iteration step 2: normres = 0.46575189394239136
[ Info: Arnoldi iteration step 3: normres = 0.30201360463938
[ Info: Arnoldi iteration step 4: normres = 1.086397369834899
[ Info: Arnoldi iteration step 5: normres = 0.3864178211866155
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.45e-01, 5.93e-02, 1.64e-01, 2.43e-01, 1.65e-02)
[ Info: Arnoldi iteration step 6: normres = 0.406190075182722
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.57e-02, 2.42e-01, 2.92e-01, 5.90e-02, 7.21e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08762074514958924
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.17e-03, 2.34e-02, 1.22e-02, 1.93e-02, 7.16e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11167132090521631
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.00e-04, 